In [68]:
import os
import numpy as np
import pandas as pd

In [69]:
year_list = ['2019', '2020', '2021', '2022', '2023']

In [71]:
shapefile_path = 'Data/London_borough_shapefile/London_Borough_Excluding_MHW.shp'
import geopandas as gpd

# 使用geopandas读取Shapefile文件
gdf = gpd.read_file(shapefile_path)

# 打印GeoDataFrame的头部
#print(gdf.head())

#print("Coordinate Reference System (CRS):")
#print(gdf.crs)

# 计算每个地区的面积（单位为平方米）
gdf['borough_area'] = gdf.geometry.area

# 打印包含面积信息的GeoDataFrame
#print(gdf[['NAME', 'borough_area']])

gdf_1 = gdf[['NAME', 'borough_area']]


## 以下代码只跑一次

In [121]:
url = ["Data/listings-2019.csv",
       "Data/listings-2020.csv",
       "Data/listings-2021.csv",
       "Data/listings-2022.csv",
       "Data/listings-2023.csv"]

In [124]:
cols_2019 = ['id', 'price', 'neighbourhood', 'availability_365', 'longitude', 'latitude', 'room_type'] #only 2019
cols = ['id', 'price', 'neighbourhood_cleansed', 'has_availability', 'longitude', 'latitude', 'host_total_listings_count',
        'room_type']

In [125]:
for year in range(0, len(year_list)):
    df = pd.read_csv(url[year], low_memory=False) # compression='gzip'
    print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")
    if year_list[year] == '2019':
        df = df[cols_2019]
        df['year'] = year_list[year]
    else:
        df = df[cols]
        df['year'] = year_list[year]
    
    path = os.path.join('Data','raw') # A default location to save raw data
    fn = url[year].split('/')[-1]
    fn = "drop_cols_" + fn 
    print(f"Writing to: {fn}")
    if not os.path.exists(path):
        print(f"Creating {path} under {os.getcwd()}")
        os.makedirs(path)
    df.to_csv(os.path.join(path,fn), index=False)
    print(f"{year} Done.")

Data frame is 85,236 x 18
Writing to: drop_cols_listings-2019.csv
0 Done.
Data frame is 76,984 x 74
Writing to: drop_cols_listings-2020.csv
1 Done.
Data frame is 70,617 x 74
Writing to: drop_cols_listings-2021.csv
2 Done.
Data frame is 69,351 x 75
Writing to: drop_cols_listings-2022.csv
3 Done.
Data frame is 87,946 x 75
Writing to: drop_cols_listings-2023.csv
4 Done.


## 以上代码只跑一次

In [126]:
url = ["Data/raw/drop_cols_listings-2019.csv",
       "Data/raw/drop_cols_listings-2020.csv",
       "Data/raw/drop_cols_listings-2021.csv",
       "Data/raw/drop_cols_listings-2022.csv",
       "Data/raw/drop_cols_listings-2023.csv"]

In [81]:
# data celaning, prepare panel data
for year in range(0, len(year_list)):
    df = pd.read_csv(url[year], low_memory=False) # compression='gzip'
    print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")
    
    if year_list[year] == '2019':
    
        df = df[df['availability_365'] != ' ']
        df['availability_365'].astype(int)
        df = df[df['availability_365'] != 0]
        
        df.drop(df[df['price'] == ' '].index, inplace=True)
        df['price'] = df['price'].astype('float')

        print(f"Data frame {year_list[year]} is {df.shape[0]:,} x {df.shape[1]}")
        
        df_new = df.groupby('neighbourhood').agg({'neighbourhood': 'count', 'price': 'mean'}).rename(columns={'neighbourhood': 'region'})
        df_new.reset_index(inplace=True)
        df_new.columns = ['borough', 'count_airbnb', 'price_airbnb']
        
        df_new['year'] = year_list[year]

        merged_df = pd.merge(df_new, gdf_1, left_on='borough', right_on='NAME', how='left')
        #merged_df.head()
        merged_df = merged_df.drop('NAME', axis=1)

        merged_df['density_airbnb'] = merged_df['count_airbnb'] / merged_df['borough_area'] * 100000
        #merged_df.head()
        merged_df['code'] = pd.factorize(merged_df['borough'])[0] + 1
        #merged_df.head()
    
    else:
        
        df = df[df['has_availability'] != 'f']

        df.drop(df[df['price'] == ' '].index, inplace=True)
        df['price'] = df['price'].str.replace('$','', regex=False).str.replace(',','', regex=False).astype('float')

        print(f"Data frame {year_list[year]} is {df.shape[0]:,} x {df.shape[1]}")
        
        df_new = df.groupby('neighbourhood_cleansed').agg({'neighbourhood_cleansed': 'count', 'price': 'mean'}).rename(columns={'neighbourhood_cleansed': 'region'})
        df_new.reset_index(inplace=True)
        df_new.columns = ['borough', 'count_airbnb', 'price_airbnb']
        
        df_new['year'] = year_list[year]

        merged_df = pd.merge(df_new, gdf_1, left_on='borough', right_on='NAME', how='left')
        #merged_df.head()
        merged_df = merged_df.drop('NAME', axis=1)

        merged_df['density_airbnb'] = merged_df['count_airbnb'] / merged_df['borough_area'] * 100000
        #merged_df.head()
        merged_df['code'] = pd.factorize(merged_df['borough'])[0] + 1
        #merged_df.head()
    
    print(df_new.info())
    print(f"Data frame is {df_new.shape[0]:,} x {df_new.shape[1]}")
    
    path = os.path.join('Data','clean') # A default location to save raw data
    fn = url[year].split('/')[-1]
    fn = "Cleaning_done_" + fn.replace("drop_cols_", "")
    print(f"Writing to: {fn}")
    if not os.path.exists(path):
        print(f"Creating {path} under {os.getcwd()}")
        os.makedirs(path)
    merged_df.to_csv(os.path.join(path,fn), index=False)
    print(f"Clean {year} Done.")


Data frame is 85,236 x 7
Data frame 2019 is 84,911 x 7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   borough       33 non-null     object 
 1   count_airbnb  33 non-null     int64  
 2   price_airbnb  33 non-null     float64
 3   year          33 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.2+ KB
None
Data frame is 33 x 4
Writing to: Cleaning_done_listings-2019.csv
Clean 0 Done.
Data frame is 76,984 x 7
Data frame 2020 is 76,984 x 7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   borough       33 non-null     object 
 1   count_airbnb  33 non-null     int64  
 2   price_airbnb  33 non-null     float64
 3   year          33 non-null     object 
dtypes: float64(1), int6

## Merge Airbnb

In [87]:
url = ["Data/clean/Cleaning_done_listings-2019.csv",
       "Data/clean/Cleaning_done_listings-2020.csv",
       "Data/clean/Cleaning_done_listings-2021.csv",
       "Data/clean/Cleaning_done_listings-2022.csv",
       "Data/clean/Cleaning_done_listings-2023.csv"]

In [91]:
# 使用列表推导式读取所有 CSV 文件并存储在一个列表中
dataframes = [pd.read_csv(file) for file in url]

# 使用 pd.concat 方法按列拼接 DataFrame
result_df = pd.concat(dataframes, axis=0)

result_df.info()

print(f"Data frame is {result_df.shape[0]:,} x {result_df.shape[1]}")

path = os.path.join('Data','final') # A default location to save raw data
fn = "Merged_airbnb.csv"
print(f"Writing to: {fn}")
if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
result_df.to_csv(os.path.join(path,fn), index=False)
print(f"Merge Done.")

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, 0 to 32
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   borough         165 non-null    object 
 1   count_airbnb    165 non-null    int64  
 2   price_airbnb    165 non-null    float64
 3   year            165 non-null    int64  
 4   borough_area    165 non-null    float64
 5   density_airbnb  165 non-null    float64
 6   code            165 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 10.3+ KB
Data frame is 165 x 7
Writing to: Merged_airbnb.csv
Merge Done.


## Rental data cleaning

In [98]:
url = ["Data/raw/london_rental_2019.xls",
       "Data/raw/london_rental_2020.xls",
       "Data/raw/london_rental_2021.xls",
       "Data/raw/london_rental_2022.xls",
       "Data/raw/london_rental_2023.xls"]

sheet_name = 'Table 1.2'

In [100]:
# data celaning, prepare rental data
for year in range(0, len(year_list)):
    
    # 读取 Excel文件中的特定工作表
    data = pd.read_excel(url[year], sheet_name=sheet_name,skiprows=12,header=0)
    specific_data = data.iloc[:198, 1:5]
    # 选择数据
    data_select = specific_data[['Borough','Count of rents','Mean']]
    data_select.rename(columns={'Count of rents': 'count_long_rents'}, inplace=True)
    # 将数据转换为数字型，去除无法转换数据
    for i in ['count_long_rents','Mean'] :
        data_select[i] = pd.to_numeric(data_select[i], errors='coerce')
        data_select = data_select.dropna(subset=[i])
     
    data_select['count_long_rents']= data_select['count_long_rents'].astype(int)
    
    data_select['TotalPrice'] = data_select['count_long_rents'] * data_select['Mean']
    
    sum_by_borough = data_select.groupby('Borough').sum()
    
    sum_by_borough['price_long_rents'] = sum_by_borough['TotalPrice'] / sum_by_borough['count_long_rents']
    sum_by_borough['Year'] = year_list[year]
    sum_by_borough['Area'] = sum_by_borough.index

    #print(df_new.info())
    #print(f"Data frame is {df_new.shape[0]:,} x {df_new.shape[1]}")
    
    path = os.path.join('Data','clean') # A default location to save raw data
    fn = url[year].split('/')[-1]
    fn = "Cleaning_done_" + fn.replace("xls", "csv")
    print(f"Writing to: {fn}")
    if not os.path.exists(path):
        print(f"Creating {path} under {os.getcwd()}")
        os.makedirs(path)
    sum_by_borough.to_csv(os.path.join(path,fn), index=False)
    print(f"Clean {year} Done.")


Writing to: Cleaning_done_london_rental_2019.csv
Clean 0 Done.
Writing to: Cleaning_done_london_rental_2020.csv
Clean 1 Done.
Writing to: Cleaning_done_london_rental_2021.csv
Clean 2 Done.
Writing to: Cleaning_done_london_rental_2022.csv
Clean 3 Done.
Writing to: Cleaning_done_london_rental_2023.csv
Clean 4 Done.


/tmp/ipykernel_505/986749759.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select.rename(columns={'Count of rents': 'count_long_rents'}, inplace=True)
/tmp/ipykernel_505/986749759.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select[i] = pd.to_numeric(data_select[i], errors='coerce')
/tmp/ipykernel_505/986749759.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

## Merge Rental

In [101]:
url = ["Data/clean/Cleaning_done_london_rental_2019.csv",
       "Data/clean/Cleaning_done_london_rental_2020.csv",
       "Data/clean/Cleaning_done_london_rental_2021.csv",
       "Data/clean/Cleaning_done_london_rental_2022.csv",
       "Data/clean/Cleaning_done_london_rental_2023.csv"]

In [102]:
# 使用列表推导式读取所有 CSV 文件并存储在一个列表中
dataframes_rental = [pd.read_csv(file) for file in url]

# 使用 pd.concat 方法按列拼接 DataFrame
result_df_rental = pd.concat(dataframes_rental, axis=0)

result_df_rental.info()

print(f"Data frame is {result_df_rental.shape[0]:,} x {result_df_rental.shape[1]}")

path = os.path.join('Data','final') # A default location to save raw data
fn = "Merged_rental.csv"
print(f"Writing to: {fn}")
if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
result_df_rental.to_csv(os.path.join(path,fn), index=False)
print(f"Merge Done.")

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, 0 to 32
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   count_long_rents  165 non-null    int64  
 1   Mean              165 non-null    float64
 2   TotalPrice        165 non-null    float64
 3   price_long_rents  165 non-null    float64
 4   Year              165 non-null    int64  
 5   Area              165 non-null    object 
dtypes: float64(3), int64(2), object(1)
memory usage: 9.0+ KB
Data frame is 165 x 6
Writing to: Merged_rental.csv
Merge Done.


## Merge airbnb and rental dataset

In [114]:
url = ["Data/final/Merged_airbnb.csv",
       "Data/final/Merged_rental.csv"]

In [115]:
df_1 = pd.read_csv(url[0], low_memory=False) # compression='gzip'
df_2 = pd.read_csv(url[1], low_memory=False) # compression='gzip'

# 使用merge方法合并DataFrame
merged_airbnb_rental = pd.merge(df_1, df_2, left_on=['borough', 'year'], right_on=['Area', 'Year'], how='left')

merged_airbnb_rental = merged_airbnb_rental.drop(['Mean', 'TotalPrice', 'Year', 'Area'], axis = 1)

# merged_airbnb_rental.set_index('year', inplace=True)

# 打印合并后的DataFrame
print(merged_airbnb_rental)

merged_airbnb_rental.info()

print(f"Data frame is {merged_airbnb_rental.shape[0]:,} x {merged_airbnb_rental.shape[1]}")

path = os.path.join('Data','final') # A default location to save raw data
fn = "Merged_airbnb_and_rental.csv"
print(f"Writing to: {fn}")
if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
merged_airbnb_rental.to_csv(os.path.join(path,fn), index=False)
print(f"Merge Done.")

                  borough  count_airbnb  price_airbnb  year  borough_area  \
0    Barking and Dagenham           372     60.529570  2019  3.610781e+07   
1                  Barnet          1652     87.562349  2019  8.674831e+07   
2                  Bexley           269     62.706320  2019  6.058023e+07   
3                   Brent          2555     93.194129  2019  4.323264e+07   
4                 Bromley           678     64.258112  2019  1.501349e+08   
..                    ...           ...           ...   ...           ...   
160                Sutton           365     90.731507  2023  4.384697e+07   
161         Tower Hamlets          6598    135.443316  2023  1.977794e+07   
162        Waltham Forest          1604    103.082918  2023  3.880797e+07   
163            Wandsworth          4203    170.175589  2023  3.426416e+07   
164           Westminster          9574    326.676311  2023  2.148698e+07   

     density_airbnb  code  count_long_rents  price_long_rents  
0          

In [127]:
url = ["Data/raw/drop_cols_listings-2019.csv",
       "Data/raw/drop_cols_listings-2020.csv",
       "Data/raw/drop_cols_listings-2021.csv",
       "Data/raw/drop_cols_listings-2022.csv",
       "Data/raw/drop_cols_listings-2023.csv"]

In [78]:
# 准备画图用的数据集
for year in range(0, len(year_list)):
    df = pd.read_csv(url[year], low_memory=False) # compression='gzip'
    print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

    if year_list[year] == '2019':
        print(df.info())
        
        df = df[df['availability_365'] != ' ']
        df['availability_365'].astype(int)
        df = df[df['availability_365'] != 0]
        
        df.drop(df[df['price'] == ' '].index, inplace=True)
        df['price'] = df['price'].astype('float')
        
        df = df[df['longitude'] != ' ']
        df['longitude'].astype(float)
        
        df = df[df['latitude'] != ' ']
        df['latitude'].astype(float)
        
        df['year'] = year_list[year]

        print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")
        
    else:
        print(df.info())
        
        df = df[df['has_availability'] != 'f']
        
        df.drop(df[df['price'] == ' '].index, inplace=True)
        df['price'] = df['price'].str.replace('$','', regex=False).str.replace(',','', regex=False).astype('float')

        df = df[df['longitude'] != ' ']
        df['longitude'].astype(float)
        
        df = df[df['latitude'] != ' ']
        df['latitude'].astype(float)
        
        df['year'] = year_list[year]

        print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")
    
    path = os.path.join('Data','clean') # A default location to save raw data
    fn = url[year].split('/')[-1]
    fn = "long_lat_Cleaning_done_" + fn.replace("drop_cols_", "")
    print(f"Writing to: {fn}")
    if not os.path.exists(path):
        print(f"Creating {path} under {os.getcwd()}")
        os.makedirs(path)
    df.to_csv(os.path.join(path,fn), index=False)
    print(f"Clean {year} Done.")

Data frame is 85,236 x 7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85236 entries, 0 to 85235
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                85236 non-null  object
 1   price             85236 non-null  object
 2   neighbourhood     85236 non-null  object
 3   availability_365  85236 non-null  object
 4   longitude         85236 non-null  object
 5   latitude          85236 non-null  object
 6   year              85236 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 4.6+ MB
None
Writing to: long_lat_Cleaning_done_listings-2019.csv
Clean 0 Done.
Data frame is 76,984 x 7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76984 entries, 0 to 76983
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      76984 non-null  int64  
 1   price                   76984 non-

In [128]:
# 准备画图用的数据集
for year in range(0, len(year_list)):
    df = pd.read_csv(url[year], low_memory=False) # compression='gzip'
    print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

    if year_list[year] == '2019':
        print(df.info())
        
        df = df[df['availability_365'] != ' ']
        df['availability_365'].astype(int)
        df = df[df['availability_365'] != 0]
        
        df.drop(df[df['price'] == ' '].index, inplace=True)
        df['price'] = df['price'].astype('float')
        
        df = df[df['longitude'] != ' ']
        df['longitude'].astype(float)
        
        df = df[df['latitude'] != ' ']
        df['latitude'].astype(float)
        
        df['year'] = year_list[year]

        print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")
        
    else:
        print(df.info())
        
        df = df[df['has_availability'] != 'f']
        
        df.drop(df[df['price'] == ' '].index, inplace=True)
        df['price'] = df['price'].str.replace('$','', regex=False).str.replace(',','', regex=False).astype('float')

        df = df[df['longitude'] != ' ']
        df['longitude'].astype(float)
        
        df = df[df['latitude'] != ' ']
        df['latitude'].astype(float)
        
        df['year'] = year_list[year]

        print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")
    
    path = os.path.join('Data','clean') # A default location to save raw data
    fn = url[year].split('/')[-1]
    fn = "room_type_Cleaning_done_" + fn.replace("drop_cols_", "")
    print(f"Writing to: {fn}")
    if not os.path.exists(path):
        print(f"Creating {path} under {os.getcwd()}")
        os.makedirs(path)
    df.to_csv(os.path.join(path,fn), index=False)
    print(f"Clean {year} Done.")

Data frame is 85,236 x 8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85236 entries, 0 to 85235
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                85236 non-null  object
 1   price             85236 non-null  object
 2   neighbourhood     85236 non-null  object
 3   availability_365  85236 non-null  object
 4   longitude         85236 non-null  object
 5   latitude          85236 non-null  object
 6   room_type         85236 non-null  object
 7   year              85236 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 5.2+ MB
None
Writing to: room_type_Cleaning_done_listings-2019.csv
Clean 0 Done.
Data frame is 76,984 x 9
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76984 entries, 0 to 76983
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         76984 n

## 测试

In [16]:
url = ["Data/raw/drop_cols_listings-2019.csv",
       "Data/raw/drop_cols_listings-2020.csv",
       "Data/raw/drop_cols_listings-2021.csv",
       "Data/raw/drop_cols_listings-2022.csv",
       "Data/raw/drop_cols_listings-2023.csv"]

In [22]:
for year in range(0, len(year_list)):
    df = pd.read_csv(url[year], low_memory=False) # compression='gzip'
    print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

    if year_list[year] == '2019':
        df = df[df['availability_365'] != ' ']
        df['availability_365'].astype(int)
        df = df[df['availability_365'] != 0]
        print(f"Data frame {year} is {df.shape[0]:,} x {df.shape[1]}")
        print(df.info())
    else:
        df = df[df['has_availability'] != 'f']
        print(f"Data frame {year} is {df.shape[0]:,} x {df.shape[1]}")
        print(df.info())

Data frame is 85,236 x 4
Data frame 0 is 84,911 x 4
<class 'pandas.core.frame.DataFrame'>
Index: 84911 entries, 0 to 85235
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             84911 non-null  object
 1   neighbourhood     84911 non-null  object
 2   availability_365  84911 non-null  object
 3   year              84911 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 3.2+ MB
None
Data frame is 76,984 x 4
Data frame 1 is 76,984 x 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76984 entries, 0 to 76983
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   price                   76984 non-null  object
 1   neighbourhood_cleansed  76984 non-null  object
 2   has_availability        76984 non-null  object
 3   year                    76984 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 

## 以下是测试

In [40]:
shapefile_path = 'Data/London_borough_shapefile/London_Borough_Excluding_MHW.shp'

In [42]:
import geopandas as gpd

# 使用geopandas读取Shapefile文件
gdf = gpd.read_file(shapefile_path)

# 打印GeoDataFrame的头部
print(gdf.head())

print("Coordinate Reference System (CRS):")
print(gdf.crs)

                   NAME   GSS_CODE   HECTARES  NONLD_AREA ONS_INNER  SUB_2009  \
0  Kingston upon Thames  E09000021   3726.117       0.000         F       NaN   
1               Croydon  E09000008   8649.441       0.000         F       NaN   
2               Bromley  E09000006  15013.487       0.000         F       NaN   
3              Hounslow  E09000018   5658.541      60.755         F       NaN   
4                Ealing  E09000009   5554.428       0.000         F       NaN   

   SUB_2006                                           geometry  
0       NaN  POLYGON ((516401.600 160201.800, 516407.300 16...  
1       NaN  POLYGON ((535009.200 159504.700, 535005.500 15...  
2       NaN  POLYGON ((540373.600 157530.400, 540361.200 15...  
3       NaN  POLYGON ((521975.800 178100.000, 521967.700 17...  
4       NaN  POLYGON ((510253.500 182881.600, 510249.900 18...  
Coordinate Reference System (CRS):
PROJCS["OSGB36 / British National Grid",GEOGCS["OSGB36",DATUM["Ordnance_Survey_of_Great_

In [48]:
# 计算每个地区的面积（单位为平方米）
gdf['borough_area'] = gdf.geometry.area

# 打印包含面积信息的GeoDataFrame
print(gdf[['NAME', 'borough_area']])

gdf_1 = gdf[['NAME', 'borough_area']]

                      NAME  borough_area
0     Kingston upon Thames  3.726118e+07
1                  Croydon  8.649444e+07
2                  Bromley  1.501349e+08
3                 Hounslow  5.592623e+07
4                   Ealing  5.554435e+07
5                 Havering  1.123497e+08
6               Hillingdon  1.157005e+08
7                   Harrow  5.046330e+07
8                    Brent  4.323264e+07
9                   Barnet  8.674831e+07
10                 Lambeth  2.681004e+07
11               Southwark  2.886204e+07
12                Lewisham  3.514908e+07
13               Greenwich  4.733407e+07
14                  Bexley  6.058023e+07
15                 Enfield  8.220019e+07
16          Waltham Forest  3.880797e+07
17               Redbridge  5.641922e+07
18                  Sutton  4.384697e+07
19    Richmond upon Thames  5.735328e+07
20                  Merton  3.762474e+07
21              Wandsworth  3.426416e+07
22  Hammersmith and Fulham  1.639762e+07
23  Kensington a

In [46]:
print(df_new.head())

                borough  count_airbnb  price_airbnb  year
0  Barking and Dagenham           536    254.712687  2023
1                Barnet          2138    159.844715  2023
2                Bexley           479     95.929019  2023
3                 Brent          2620    182.354962  2023
4               Bromley           742    107.086253  2023


In [50]:
merged_df = pd.merge(df_new, gdf_1, left_on='borough', right_on='NAME', how='left')
merged_df.head()
merged_df = merged_df.drop('NAME', axis=1)

In [59]:
merged_df['density_airbnb'] = merged_df['count_airbnb'] / merged_df['borough_area'] * 100000
merged_df.head()
merged_df['code'] = pd.factorize(merged_df['borough'])[0] + 1
merged_df.head()

,borough,count_airbnb,price_airbnb,year,borough_area,density_airbnb,code
0,Barking and Dagenham,536,254.712687,2023,3.610781e+07,1.484443,1
1,Barnet,2138,159.844715,2023,8.674831e+07,2.464601,2
2,Bexley,479,95.929019,2023,6.058023e+07,0.790687,3
3,Brent,2620,182.354962,2023,4.323264e+07,6.060236,4
4,Bromley,742,107.086253,2023,1.501349e+08,0.494222,5
